In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/6962 final project/data/precip_and_veg.csv')

In [ ]:
df = df.drop(columns=['Unnamed: 0','year'		,'half'	,'cor'])

In [ ]:
df

,time,lat,lon,precip,target_label
0,03/01/2000,-34,18,0.000000,0
1,03/01/2000,-33,26,0.065625,0
2,03/01/2000,-31,21,0.000000,0
3,03/01/2000,-30,22,0.026250,0
4,03/01/2000,-29,25,0.033750,0
...,...,...,...,...,...
970825,12/31/2019,47,42,0.000000,0
970826,12/31/2019,48,41,0.000000,0
970827,12/31/2019,48,44,0.112500,0
970828,12/31/2019,48,66,2.415755,0


#Feature Creation

In [ ]:
df_models = df.copy()
df_models['time'] = pd.to_datetime(df_models['time'])

'alerted' Whether the area around has had any alert before

In [ ]:
df_models.head()

,time,lat,lon,precip,target_label,veg
0,2000-03-01,-34,18,0.000000,0,2783.154098
1,2000-03-01,-33,26,0.065625,0,5935.667500
2,2000-03-01,-31,21,0.000000,0,2043.272500
3,2000-03-01,-30,22,0.026250,0,3039.685000
4,2000-03-01,-29,25,0.033750,0,4036.415000


In [ ]:
df_models = df_models.sort_values(by=['lat','lon'])

In [ ]:
df_models.head()

,time,lat,lon,precip,target_label,veg
0,2000-03-01,-34,18,0.000000,0,2783.154098
134,2000-03-02,-34,18,0.000000,0,2783.154098
268,2000-03-03,-34,18,0.052500,0,2783.154098
402,2000-03-04,-34,18,0.031875,0,2783.154098
536,2000-03-05,-34,18,0.005625,0,2783.154098


In [ ]:
#each location has 7245 rows
for i in range(1,91):
  df_models['p_'+str(i)] = df_models['precip'].shift(+i)

In [ ]:
df_models.head()

,time,lat,lon,precip,target_label,veg,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,...,p_51,p_52,p_53,p_54,p_55,p_56,p_57,p_58,p_59,p_60,p_61,p_62,p_63,p_64,p_65,p_66,p_67,p_68,p_69,p_70,p_71,p_72,p_73,p_74,p_75,p_76,p_77,p_78,p_79,p_80,p_81,p_82,p_83,p_84,p_85,p_86,p_87,p_88,p_89,p_90
0,2000-03-01,-34,18,0.000000,0,2783.154098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,2000-03-02,-34,18,0.000000,0,2783.154098,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,2000-03-03,-34,18,0.052500,0,2783.154098,0.000000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,2000-03-04,-34,18,0.031875,0,2783.154098,0.052500,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,2000-03-05,-34,18,0.005625,0,2783.154098,0.031875,0.0525,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(1,91):
  df_models = df_models.loc[-df_models['time'].isin([(dt.datetime(2000, 3, 1) + timedelta(days = i)) for i in range(0,91) ])]

df_models = df_models.sort_values(by=['lat','lon'])
df_models.reset_index(drop=True,inplace=True)

#Deprecated

In [ ]:
# df_groups['coordinate'] = df_groups.apply(lambda x: (x['lat'], x['lon']),axis=1)

In [ ]:
def earliest_alert(_df):
  _df = _df.copy()
  date_set = set(_df.date)
  cor_dict = dict.fromkeys(set(_df.coordinate),list()) #gives a list of all alert dates at each coordinate
  for i in set(_df.coordinate):
    cor_dict[i] = ((((_df.loc[_df.coordinate == i]).loc[(_df.loc[_df.coordinate == i]).label == 1]).date).to_list())
  earliest_alert_dict = dict()
  for key, value in cor_dict.items():
    if len(value):
      earliest_alert_dict[key] = min(value)
  return earliest_alert_dict

In [ ]:
# earliest_alert_dates = earliest_alert(df_models)
# earliest_alert_dates

In [ ]:
def alerted(_x, _earliest_alert_dates):
  if _x.coordinate not in _earliest_alert_dates.keys():
    return 0
  elif _x.date > _earliest_alert_dates[_x.coordinate]:
    return 1
  else:
    return 0

In [ ]:
# df_models['alerted'] = df_models.apply(lambda x: alerted(x, earliest_alert_dates), axis = 1 )

#7day_avg, 14day_avg, 30day_avg,3month_avg



In [ ]:
df_models['avg_1_3'] = (df_models['p_1'] + df_models['p_2'] + df_models['p_3'])/3
df_models['avg_5_6'] = (df_models['p_5'] + df_models['p_6'])/2
df_models['7day_avg'] = df_models['p_1'] + df_models['p_2'] + df_models['p_3'] + df_models['p_4'] + df_models['p_5'] 

a = df_models['p_1'].copy()
for i in range(2,8):
  a += (df_models['p_'+str(i)])
df_models['7day_avg'] = a/7

a = df_models['p_1'].copy()
for i in range(2,31):
  a += (df_models['p_'+str(i)])
df_models['1month_avg'] = a/30

a = df_models['p_1'].copy()
for i in range(2,61):
  a += (df_models['p_'+str(i)])
df_models['2month_avg'] = a/60

a = df_models['p_1'].copy()
for i in range(2,91):
  a += (df_models['p_'+str(i)])
df_models['3month_avg'] = a/90

In [ ]:
df_models.reset_index(inplace=True,drop=True)

In [ ]:
(df_models.columns)

Index(['time', 'lat', 'lon', 'precip', 'target_label', 'veg', 'p_1', 'p_2',
       'p_3', 'p_4',
       ...
       'p_87', 'p_88', 'p_89', 'p_90', 'avg_1_3', 'avg_5_6', '7day_avg',
       '1month_avg', '2month_avg', '3month_avg'],
      dtype='object', length=102)

In [ ]:
df_ = df_models.drop(['p_'+str(i) for i in range(15,91)], axis=1)

In [ ]:
%cd /content/drive/MyDrive/6962 final project/data/

/content/drive/.shortcut-targets-by-id/12pjah520zwJMQos4SHRxl7lFoUlEuU7h/6962 final project/data


In [ ]:
df_

,time,lat,lon,precip,target_label,veg,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,avg_1_3,avg_5_6,7day_avg,1month_avg,2month_avg,3month_avg
0,2000-05-31,-34,18,6.973125,0,2783.154098,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,0.026250,2.030625,2.622187,2.934375,2.817937,1.425406,0.959792
1,2000-06-01,-34,18,15.020625,0,2783.154098,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,4.282500,5.076562,3.484286,3.049187,1.541625,1.037271
2,2000-06-02,-34,18,1.725000,0,2783.154098,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,8.316250,3.149062,5.462679,3.347937,1.791969,1.203583
3,2000-06-03,-34,18,2.235000,0,2783.154098,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,7.906250,1.568437,5.127321,3.397125,1.819844,1.222396
4,2000-06-04,-34,18,2.100000,0,2783.154098,2.235000,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,6.326875,2.937188,4.577946,3.454937,1.857094,1.247167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958631,2019-12-27,49,43,0.684375,0,3646.470000,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.000000,0.016875,0.000000,0.007232,1.102688,1.526656,2.969875
958632,2019-12-28,49,43,0.000000,0,3646.470000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.228125,0.000000,0.105000,1.125438,1.538063,2.977479
958633,2019-12-29,49,43,0.000000,0,3646.470000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375
958634,2019-12-30,49,43,3.502500,0,3646.470000,0.000000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375


In [ ]:
df_.to_csv('final_csv_with_features.csv')

#Precipitation plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
df_models = df_

In [ ]:
df_models['cor'] = df_models.apply(lambda x: (x['lat'],x['lon']),axis=1)

In [ ]:
df_models.groupby('cor').agg({'target_label':'sum'}).sort_values('target_label')

,target_label
cor,
"(17, -4)",0
"(14, -5)",0
"(-34, 18)",1
"(31, 46)",1
"(31, -7)",1
...,...
"(47, 42)",41
"(26, 68)",42
"(48, 66)",42


In [ ]:
random.seed(0)
sample_df = df_models.loc[ df_models['cor'] == (29, 69)			] 
sample_df['precip'] = StandardScaler().fit_transform(np.array(sample_df[['precip']]))
sample_df.loc[sample_df.target_label == 1].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,time,lat,lon,precip,target_label,veg,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,avg_1_3,avg_5_6,7day_avg,1month_avg,2month_avg,3month_avg,cor
479437,2000-09-27,29,69,-0.568994,1,1177.445,0.091875,0.000000,1.313125,1.995000,0.100000,0.246875,0.000000,2.252500,4.202500,0.541875,1.15750,0.318125,17.899376,0.000000,0.468333,0.173437,0.535268,1.970333,1.802073,1.750826,"(29, 69)"
479485,2000-11-14,29,69,-0.287969,1,1177.445,1.144375,0.000625,0.000000,0.006875,1.205625,0.051875,2.061875,5.510000,4.858125,1.251875,2.23500,0.000000,0.041875,1.151875,0.381667,0.628750,0.638750,1.688437,1.315521,1.572840,"(29, 69)"
479493,2000-11-22,29,69,-0.531132,1,1177.445,3.126875,1.170000,6.378750,0.000000,4.230000,0.025625,0.350625,0.940625,1.144375,0.000625,0.00000,0.006875,1.205625,0.051875,3.558542,2.127813,2.183125,1.715021,1.438906,1.590396,"(29, 69)"
479622,2001-03-31,29,69,0.356156,1,1140.110,0.003750,0.035625,0.000000,0.000000,0.000000,2.071250,1.231250,0.302500,3.939375,1.047500,0.86625,1.947500,3.635625,0.000000,0.013125,1.035625,0.477411,2.114542,1.841833,1.775382,"(29, 69)"
479626,2001-04-04,29,69,-0.483628,1,1140.110,0.067500,1.660625,2.001875,2.652500,0.003750,0.035625,0.000000,0.000000,0.000000,2.071250,1.23125,0.302500,3.939375,1.047500,1.243333,0.019687,0.917411,2.199667,1.773521,1.605285,"(29, 69)"


In [ ]:
# sns.set(rc = {'figure.figsize':(30,8)})
# sns.lineplot(data=sample_df, x="time", y="precip", hue="cor")

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig =  px.scatter(sample_df.loc[sample_df.target_label == 8], x="time", y="target_label",
#                  title="Gender Earnings Disparity",
#                  labels={"target_label":"target_label"}, # customize axis label,
#                   color="target_label"
#                 )

fig.add_trace(
    go.Scatter(x=list(sample_df.loc[sample_df['target_label'] != 0].time- timedelta(days=0)), y=list(sample_df.loc[sample_df['target_label'] != 0].target_label), 
               mode='markers',line=dict(color='rgb(255,0,0)'),
               name = 'CCHF Alert'
               )
    )

fig.update_yaxes(ticks="outside", tickwidth=2, tickcolor='crimson',range = [-4,4],secondary_y=False)
fig.add_trace(
    go.Scatter(x=list(sample_df.time), y=list(sample_df['1month_avg']),
               line=dict(color='rgb(0,0,255)'),
              name = 'Precipitation'
              ),
    secondary_y=True
    )
# fig.add_trace(
  # go.Scatter(x=list(sample_df.time), y=list(sample_df.target_label)))

# Set title
fig.update_layout(
    title_text="Precipitation Plot"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()


#Logistic Regression

In [ ]:
df_models.columns

Index(['time', 'lat', 'lon', 'precip', 'target_label', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'p_6', 'p_7', 'p_8', 'p_9', 'p_10', 'p_11', 'p_12',
       'p_13', 'p_14', 'avg_1_3', 'avg_5_6', '7day_avg', '1month_avg',
       '2month_avg', '3month_avg', 'cor'],
      dtype='object')

In [ ]:
df_models

,time,lat,lon,precip,target_label,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,avg_1_3,avg_5_6,7day_avg,1month_avg,2month_avg,3month_avg,cor
0,2000-05-31,-34,18,6.973125,0,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,0.026250,2.030625,2.622187,2.934375,2.817937,1.425406,0.959792,"(-34, 18)"
1,2000-06-01,-34,18,15.020625,0,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,4.282500,5.076562,3.484286,3.049187,1.541625,1.037271,"(-34, 18)"
2,2000-06-02,-34,18,1.725000,0,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,8.316250,3.149062,5.462679,3.347937,1.791969,1.203583,"(-34, 18)"
3,2000-06-03,-34,18,2.235000,0,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,7.906250,1.568437,5.127321,3.397125,1.819844,1.222396,"(-34, 18)"
4,2000-06-04,-34,18,2.100000,0,2.235000,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,6.326875,2.937188,4.577946,3.454937,1.857094,1.247167,"(-34, 18)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958631,2019-12-27,49,43,0.684375,0,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.000000,0.016875,0.000000,0.007232,1.102688,1.526656,2.969875,"(49, 43)"
958632,2019-12-28,49,43,0.000000,0,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.228125,0.000000,0.105000,1.125438,1.538063,2.977479,"(49, 43)"
958633,2019-12-29,49,43,0.000000,0,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375,"(49, 43)"
958634,2019-12-30,49,43,3.502500,0,0.000000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375,"(49, 43)"


In [ ]:
df_logi = df_models.copy()

In [ ]:
df_logi.columns

Index(['time', 'lat', 'lon', 'precip', 'target_label', 'p_1', 'p_2', 'p_3',
       'p_4', 'p_5', 'p_6', 'p_7', 'p_8', 'p_9', 'p_10', 'p_11', 'p_12',
       'p_13', 'p_14', 'avg_1_3', 'avg_5_6', '7day_avg', '1month_avg',
       '2month_avg', '3month_avg', 'cor'],
      dtype='object')

In [ ]:
df_logi.drop(labels=['time', 'lat', 'lon','target_label','cor'],axis = 1).astype('float32' )

,precip,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,avg_1_3,avg_5_6,7day_avg,1month_avg,2month_avg,3month_avg
0,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,0.026250,2.030625,2.622187,2.934375,2.817937,1.425406,0.959792
1,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,4.282500,5.076562,3.484286,3.049187,1.541625,1.037271
2,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,8.316250,3.149062,5.462678,3.347937,1.791969,1.203583
3,2.235000,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,7.906250,1.568437,5.127321,3.397125,1.819844,1.222396
4,2.100000,2.235000,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,6.326875,2.937187,4.577947,3.454937,1.857094,1.247167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958631,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.000000,0.016875,0.000000,0.007232,1.102687,1.526656,2.969875
958632,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.228125,0.000000,0.105000,1.125437,1.538062,2.977479
958633,0.000000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,0.228125,0.025313,0.105000,1.125437,1.490375,2.977375
958634,3.502500,0.000000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.228125,0.025313,0.105000,1.125437,1.490375,2.977375


In [ ]:
df_logi

,time,lat,lon,precip,target_label,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,avg_1_3,avg_5_6,7day_avg,1month_avg,2month_avg,3month_avg,cor
0,2000-05-31,-34,18,6.973125,0,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,0.026250,2.030625,2.622187,2.934375,2.817937,1.425406,0.959792,"(-34, 18)"
1,2000-06-01,-34,18,15.020625,0,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,0.215625,4.282500,5.076562,3.484286,3.049187,1.541625,1.037271,"(-34, 18)"
2,2000-06-02,-34,18,1.725000,0,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,0.481875,8.316250,3.149062,5.462679,3.347937,1.791969,1.203583,"(-34, 18)"
3,2000-06-03,-34,18,2.235000,0,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,0.695625,7.906250,1.568437,5.127321,3.397125,1.819844,1.222396,"(-34, 18)"
4,2000-06-04,-34,18,2.100000,0,2.235000,1.725000,15.020625,6.973125,2.955000,2.919375,0.217500,6.080625,4.072500,1.171875,3.123750,23.443125,19.336874,5.758125,6.326875,2.937188,4.577946,3.454937,1.857094,1.247167,"(-34, 18)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958631,2019-12-27,49,43,0.684375,0,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.000000,0.016875,0.000000,0.007232,1.102688,1.526656,2.969875,"(49, 43)"
958632,2019-12-28,49,43,0.000000,0,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,4.218750,0.228125,0.000000,0.105000,1.125438,1.538063,2.977479,"(49, 43)"
958633,2019-12-29,49,43,0.000000,0,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375,"(49, 43)"
958634,2019-12-30,49,43,3.502500,0,0.000000,0.000000,0.684375,0.000000,0.000000,0.050625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.871875,0.000000,0.228125,0.025313,0.105000,1.125438,1.490375,2.977375,"(49, 43)"


In [ ]:
import random
random.seed(0)
alert_idx = df_logi.loc[df_logi['target_label'] != 0].index
sample_idx = random.sample(list(df_logi.index),len(alert_idx)
X = (df_logi.drop(labels=['lon','lat','time', 'target_label','cor'],axis = 1).iloc[sample_idx]).append(df_logi.drop(labels=['lon','lat','time', 'target_label','cor'],axis = 1).iloc[alert_idx])
y = (df_logi['target_label'].iloc[sample_idx]).append(df_logi['target_label'].iloc[alert_idx])

In [ ]:
X.reset_index(inplace=True,drop=True)
y.reset_index(inplace=True,drop=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn import metrics
from sklearn.metrics import accuracy_score

# predictors = df_logi[[ 'avg_1_3', 'avg_5_6','1month_avg', '3month_avg']]
# target = df_logi['target_label']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.33, random_state = 0)


clf = LogisticRegression(random_state=0).fit(X_train, y_train.astype(int))
y_pred = clf.predict(X_test).astype(int)
y_test = y_test.astype(int)

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

accuracy_score(y_test, y_pred)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



0.4805914972273567

In [ ]:
y_2 = y.copy()

In [ ]:
y_2[(y_2.loc[(y_2>1)]).index] = 1

In [ ]:
import statsmodels.api as sm
import numpy as np
logit_model=sm.Logit(np.asarray(y_2),X,solver='liblinear')
result=logit_model.fit()
print(result.summary2())

         Current function value: 0.688517
         Iterations: 35
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.007    
Dependent Variable:   y                  AIC:                2291.5815
Date:                 2021-12-07 09:04   BIC:                2388.8037
No. Observations:     1638               Log-Likelihood:     -1127.8  
Df Model:             17                 LL-Null:            -1135.4  
Df Residuals:         1620               LLR p-value:        0.58404  
Converged:            0.0000             Scale:              1.0000   
No. Iterations:       35.0000                                         
----------------------------------------------------------------------
            Coef.    Std.Err.     z    P>|z|     [0.025       0.975]  
----------------------------------------------------------------------
precip      0.0024      0.0113  0.2157 0.8293      -0.0197      0.0245
p_1        -0.0080         nan     nan 

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning:

invalid value encountered in sqrt

